<a href="https://colab.research.google.com/github/barudenko/projects/blob/main/gectaro_research/gectaro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Исследовательский анализ данных сервиса для управления строительством и ремонтом Gectaro

__Описание проекта:__  
Gectaro – облачный сервис для автоматизации управления строительством и ремонтом.
Компания продает софт с платной подпиской в сегменте B2B (наиболее частый срок подписки 12-24 месяца), предназначенный для управления процессами в сфере строительства и ремонта.
Основная проблема – это уход из сервиса большого количества подписчиков и не продление подписки.


__Цель проекта:__  

Проанализировать имеющиеся данные, составить портреты наиболее типичных клиентов, разобраться в причинах оттока клиентов, предложить решения по их минимизации.


__План исследования:__  
1. Провести исследовательский анализ данных;
2. Сформулировать и проверить гипотезы, основываясь на имеющихся признаках;
3. Формулирование выводов по полученным результатам, подготовка презентации.

__Описание данных__  
Для работы вам представлено 4 файла с описанием компаний и их сотрудников: роли, количество работающих в системе, количество совершенных действий по месяцам и прочая информация.

**Файл активностей `activities.csv`**. Сгруппирован по компаниям, отсортирован по компаниям, а в пределах каждой компании - по месяцам

`Company_id` — Идентификатор компании  
`Month` — Отчетный месяц  
`Company-month-billing` — Количество счетов, выставленных в приложении, компанией за месяц  
`Company-month-stocks` — Месячное количество складов (аренда, заказы и т.д.), используемых компанией  
`Company-month-backups` — Месячное количество бэкапов компании  
`Company-month-projects` — Месячное количество проектов компании  
`Company-month-contractors` — Месячное количество контрагентов компании  
`Company-month-resource_orders` — Месячное количество заказов материалов компании

**Файл платежей `billing.csv`**

`Unnamed:0` — Старый индекс   
`User_id` — Идентификатор пользователя  
`Company_id` — Идентификатор компании пользователя  
`Billing_tariff_id` — Тарифный план  
*тарифы 1,8, 22 - тестовые, бесплатные и пробные. Если такие есть - при анализе не должны использоваться. для остальных тарифов - основное различие - разное количество пользователей  
тарифы 4, 7, 8, 11, 19 - группа “старт”  
тарифы 2, 17 - группа “базовый”  
тарифы 9, 20 - группа “бизнес”  
тарифы 3, 10, 13, 21 - группа “Профи”*  
`Payed_until` — Дата, до которой подписка компании действует  
`Is_payed` — Статус оплаты  
`Created_at` — Дата создания  
`Updated_at` — Дата обновления  
`Billing_tariff_price_id` — id стоимости тарифа  
*В разное время или для разных клиентов  один и тот же тариф может стоить по-разному*  
`Comment` — Комментарии  
`Pay_date` — Дата платежа

**Файл компаний `companies.csv`**

`Id` — id компании  
`Working_direction` — сфера деятельности  
`Employer_count_interval` — id количества сотрудников  
*По возрастающей id от 1 до 8, количество сотрудников - (1-4, 5-10, 11-20, 21-30, 31-50, 51-100, 101-200, >201)*  
`Created_at` — дата создания компании в системе  
`Updated_at` — дата обновления информации о компании  
`Billing_tariff_id` — id тарифа  
*см. выше*  
`Billing_is_over` — истекла ли оплата

**Файл сотрудников `people.csv`**

`Id` — индекс  
`Company_id` — id компании  
`User_id` — id пользователя  
`Role_id` — id роли (должности)  
`Created_at` — дата создания пользователя  
`Updated_at` — дата обновления информации о пользователе  
`Is_deleted` — удален ли пользователь  
`Deleted_at` — дата удаления

# Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import iqr
from scipy.stats import chi2_contingency
from scipy.stats import ttest_ind
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from datetime import date

In [2]:
#отключение текста предупреждений pandas
pd.options.mode.chained_assignment = None

#параметры отображения графиков
# pio.renderers.default='notebook'
# pio.renderers.default='colab'

In [3]:
activities, billing, companies, people = (
    pd.read_csv('https://drive.google.com/uc?export=download&id=19Y3STAR7-JMeFWqFxwCeV_cfbdcpywVA', index_col=0),
    pd.read_csv('https://drive.google.com/uc?export=download&id=1HmxDJg7JBrlK7zcif5O9EpPekw_obCZf', index_col=0),
    pd.read_csv('https://drive.google.com/uc?export=download&id=1F4won1jNoFDgvz7nKYRlKXv9OKrDtItb', index_col=0),
    pd.read_csv('https://drive.google.com/uc?export=download&id=1PCiXos1cNUGwBrha3VSeZtiPvyNAESTd', index_col=0)
)

In [4]:
display(activities.head(3))
activities.shape

,company_id,month,company-month-billing,company-month-stocks,company-month-backups,company-month-projects,company-month-contractors,company-month-resource_orders
0,1,2019-09-01 00:00:00+00:00,1.0,0.0,0.0,7.0,0.0,0.0
1,1,2019-12-01 00:00:00+00:00,1.0,0.0,0.0,2.0,1.0,0.0
2,1,2020-04-01 00:00:00+00:00,4.0,1.0,0.0,1.0,6.0,0.0


(6316, 8)

In [5]:
display(billing.head(3))
billing.shape

,Unnamed: 0,id,user_id,company_id,billing_tariff_id,payed_until,is_payed,created_at,updated_at,billing_tariff_price_id,comment,pay_date
4,4,3050,567,512,3,1640153302,False,1608617302,1608617302,12.0,NaN,NaN
6,6,3051,2861,2474,7,1640131200,True,1608626162,1608626181,23.0,NaN,1.608626e+09
16,16,3059,1902,1645,2,1616577484,False,1608801484,1608801484,6.0,NaN,NaN


(3480, 12)

In [6]:
display(companies.head(3))
companies.shape

,id,working_direction,employer_count_interval,created_at,updated_at,billing_tariff_id,billing_is_over
19,514,-,1.0,1571316635,1670298314,4.0,True
57,383,Строительство,2.0,1564142565,1676550050,7.0,False
69,495,-,1.0,1570535695,1598281714,2.0,False


(723, 7)

In [7]:
display(people.head(3))
people.shape

,id,company_id,user_id,role_id,created_at,updated_at,is_deleted,deleted_at
2,6601,495,NaN,7133,1570535962,1570535962,True,1.594067e+09
6,7025,1,NaN,1,1594120484,1594120484,True,1.594120e+09
7,7165,1255,1734.0,18085,1595438102,1595439391,True,1.633960e+09


(5095, 8)

Так как pandas при загрузке датасетов присваивает новые индексы, загрузил все датасеты с параметром `index_col=0`, они идут не по порядку т.к. данные изначально отсортированы, после предобработки ресетну их.

Дальше предобработка

# Предобработка

## activities

In [8]:
activities.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6316 entries, 0 to 6556
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   company_id                     6316 non-null   int64  
 1   month                          6316 non-null   object 
 2   company-month-billing          6316 non-null   float64
 3   company-month-stocks           6316 non-null   float64
 4   company-month-backups          6316 non-null   float64
 5   company-month-projects         6316 non-null   float64
 6   company-month-contractors      6316 non-null   float64
 7   company-month-resource_orders  6316 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 444.1+ KB


In [9]:
#полные дубликаты
activities.duplicated().sum()

0

In [10]:
#неполные дубликаты компания-месяц
activities[['company_id', 'month']].duplicated().sum()

0

In [11]:
#пропуски
activities.isna().mean()

,0
company_id,0.0
month,0.0
company-month-billing,0.0
company-month-stocks,0.0
company-month-backups,0.0
company-month-projects,0.0
company-month-contractors,0.0
company-month-resource_orders,0.0


In [12]:
#типы данных
activities['month'] = pd.to_datetime(activities.month).dt.date

In [13]:
#переименовываю столбцы
activities.columns = [x.replace('company-month-', '') for x in activities.columns]

In [14]:
#опять типы данных
for col in activities.columns[2:]:
  activities[col] = activities[col].astype('int')

In [15]:
#сбрасываю индексы
activities.reset_index(drop=True, inplace=True)

## billing

In [16]:
billing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3480 entries, 4 to 18953
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               3480 non-null   int64  
 1   id                       3480 non-null   int64  
 2   user_id                  3480 non-null   int64  
 3   company_id               3480 non-null   int64  
 4   billing_tariff_id        3480 non-null   int64  
 5   payed_until              3480 non-null   int64  
 6   is_payed                 3480 non-null   bool   
 7   created_at               3480 non-null   int64  
 8   updated_at               3480 non-null   int64  
 9   billing_tariff_price_id  3480 non-null   float64
 10  comment                  745 non-null    object 
 11  pay_date                 1108 non-null   float64
dtypes: bool(1), float64(2), int64(8), object(1)
memory usage: 329.6+ KB


In [17]:
#сравниваем индексы со столбцом 'Unnamed: 0'
np.array_equal(np.array(billing['Unnamed: 0']), billing.index.values)

True

In [18]:
#дропаем столбцы Unnamed: 0 и id
billing.drop(['Unnamed: 0', 'id'], axis=1, inplace=True)

Сразу дропнул колонки `Unnamed: 0` и `id`, первая дублируется с индексом, вторая бесполезная

In [19]:
#полные дубликаты
billing.duplicated().sum()

1

In [20]:
#один, дропнем его сразу
billing.drop_duplicates(inplace=True)

In [21]:
#неполные по компании, дате выставления счета и дате оплаты
billing[['pay_date', 'company_id','created_at']].duplicated().sum()

0

In [22]:
#пропуски
billing.isna().mean()

,0
user_id,0.000000
company_id,0.000000
billing_tariff_id,0.000000
payed_until,0.000000
is_payed,0.000000
created_at,0.000000
updated_at,0.000000
billing_tariff_price_id,0.000000
comment,0.785858
pay_date,0.681518


In [23]:
#преобразовываю типы данных
billing['payed_until'] = pd.to_datetime(billing.payed_until, unit='s')
billing['created_at'] = pd.to_datetime(billing.created_at, unit='s')
billing['updated_at'] = pd.to_datetime(billing.updated_at, unit='s')

In [24]:
billing['pay_date'] = pd.to_datetime(billing.pay_date, unit='s', errors='ignore')

<ipython-input-24-fc6e5ce1e0db>:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  billing['pay_date'] = pd.to_datetime(billing.pay_date, unit='s', errors='ignore')


In [25]:
# for col in ['billing_tariff_id', 'billing_tariff_price_id', 'user_id', 'company_id']:
#   billing[col] = billing[col].astype('int')

for col in ['billing_tariff_id', 'user_id', 'company_id']:
  billing[col] = billing[col].astype('int')
billing['billing_tariff_price_id'] = billing.billing_tariff_price_id.astype('int').astype('str')

In [26]:
billing.created_at.duplicated().sum()

0

В датах созданных платежей нет дубликатов, значит ситуация, когда счет на оплату выставляется в одной строке, и оплачивается позже в другой строке, не наблюдается. Тогда не понятно, какая информация содержится в поле `updated_at`.

In [27]:
billing[billing.is_payed == True]['pay_date'].isna().sum()

245

245 строк с оплатой, но без даты. Остальные пропуски в комментариях и в дате оплаты в неоплаченых счетах

In [28]:
#компании с тестовыми, бесплатными и пробными тарифами
billing[billing.billing_tariff_id.isin([1, 18, 22])].shape[0]

0

Компаний с тестовыми, бесплатными и пробными тарифами в датасете нет

In [29]:
#сбрасываю индексы
billing.reset_index(drop=True, inplace=True)

## companies

In [30]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 723 entries, 19 to 17972
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       723 non-null    int64  
 1   working_direction        720 non-null    object 
 2   employer_count_interval  723 non-null    float64
 3   created_at               723 non-null    int64  
 4   updated_at               723 non-null    int64  
 5   billing_tariff_id        716 non-null    float64
 6   billing_is_over          723 non-null    bool   
dtypes: bool(1), float64(2), int64(3), object(1)
memory usage: 40.2+ KB


In [31]:
#полные дубликаты
companies.duplicated().sum()

0

In [32]:
#пропуски
companies.isna().mean()

,0
id,0.000000
working_direction,0.004149
employer_count_interval,0.000000
created_at,0.000000
updated_at,0.000000
billing_tariff_id,0.009682
billing_is_over,0.000000


In [33]:
#в поле working_direction меняю пропуски на '-'
companies.working_direction.replace(np.nan, '-', inplace=True)

<ipython-input-33-7bdc3221000a>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  companies.working_direction.replace(np.nan, '-', inplace=True)


In [34]:
#в поле billing_tariff_id пропусков меньше процента, это тарифный план, заменять чем-то бессмысленно, дропаю их
companies.dropna(subset='billing_tariff_id', inplace=True)

In [35]:
#меняем типы данных
companies['employer_count_interval'] = companies.employer_count_interval.astype('int')
companies['billing_tariff_id'] = companies.billing_tariff_id.astype('int')

In [36]:
companies['created_at'] = pd.to_datetime(companies.created_at, unit='s')
companies['updated_at'] = pd.to_datetime(companies.updated_at, unit='s')

In [37]:
#неполные дубликаты по компаниям
companies.id.duplicated().sum()

0

In [38]:
companies.id.nunique() == len(companies)

True

Все компании уникальны

In [39]:
#сбрасываю индексы
companies.reset_index(drop=True, inplace=True)

## people

In [40]:
people.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5095 entries, 2 to 17054
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          5095 non-null   int64  
 1   company_id  5095 non-null   int64  
 2   user_id     3813 non-null   float64
 3   role_id     5095 non-null   int64  
 4   created_at  5095 non-null   int64  
 5   updated_at  5095 non-null   int64  
 6   is_deleted  5095 non-null   bool   
 7   deleted_at  2180 non-null   float64
dtypes: bool(1), float64(2), int64(5)
memory usage: 323.4 KB


In [41]:
#дропаем колонку id, она бесполезная
people.drop('id', axis=1, inplace=True)

In [42]:
#дубликаты полные
people.duplicated().sum()

7

In [43]:
#дропаем их
people.drop_duplicates(inplace=True)

In [44]:
#дубликаты неудаленных пользователей с одним айдишником в одной компании на одной должности
#people[people['is_deleted'] == False][['company_id', 'user_id', 'role_id']].duplicated().sum()

In [45]:
#пропуски
people.isna().mean()

,0
company_id,0.000000
user_id,0.250590
role_id,0.000000
created_at,0.000000
updated_at,0.000000
is_deleted,0.000000
deleted_at,0.571737


Четверть пропусков в `user_id`, заменю их нулями, дропать так много не хочется

In [46]:
people.user_id.replace(np.nan, 0, inplace=True)

<ipython-input-46-b8a89c55317b>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  people.user_id.replace(np.nan, 0, inplace=True)


In [47]:
#меняю типы данных
people['created_at'] = pd.to_datetime(people.created_at, unit='s')
people['updated_at'] = pd.to_datetime(people.updated_at, unit='s')

In [48]:
people['deleted_at'] = pd.to_datetime(people.deleted_at, unit='s', errors='ignore')

<ipython-input-48-d10f9d17df16>:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  people['deleted_at'] = pd.to_datetime(people.deleted_at, unit='s', errors='ignore')


In [49]:
people['user_id'] = people.user_id.astype('int')

In [50]:
#сбрасываю индексы
people.reset_index(drop=True, inplace=True)

# Исследовательский анализ

In [51]:
#задаю дату исследования
research_date = date(2024, 3, 28)
research_date

datetime.date(2024, 3, 28)

In [52]:
#первая дата записи платежа
billing.created_at.min()

Timestamp('2019-07-23 09:32:39')

In [53]:
#первая дата записи о компании
companies.created_at.min()

Timestamp('2019-01-23 14:00:51')

In [54]:
#первая дата записи активности компании
activities.month.min()

datetime.date(2019, 1, 1)

Имеются данные с начала 2019 года, платежи начались в июле 2019 года.

In [55]:
#количество уникальных компаний в billing
billing.company_id.nunique()

1841

In [56]:
#количество уникальных компаний в companies
companies.id.nunique()

716

В датасете платежей `billing` 1841 компания, в датасете с информацией о компаниях `companies` всего 716

In [57]:
#пересечение компаний с датасетом companies
len(set(companies.id) & (set(billing.company_id)))

716

В пересечении тоже 716.

In [58]:
#айдишники когда-либо плативших компаний
paying_companies_set = set(billing.query('is_payed == True')['company_id'])

In [59]:
len(paying_companies_set)

779

Когда-либо плативших компаний 779, из них не все есть в `companies` что говорит о плохом качестве данных

In [60]:
#количество уникальных компаний в activities
activities.company_id.nunique()

716

Та же ситуация и с `activities` - всего 716 компаний. Данные о клиентах у сервиса не полные.

Если оставлю компании, которые есть в `companies` и `activities`, то потеряю больше половины данных.  
Пока буду работать с датасетом `billing`.

## Данные о платежах

Присвою каждой компании статус активна/неактивна по признаку даты действия подписки. Если дата подписки на сервис меньше даты исследования будем считать компанию прекратившей пользоваться сервисом(неактивной). Если больше - компания все еще активный клиент.

In [61]:
#группирую billing по компаниям, беру наибольшую дату подписки
companies_by_subscription_date = billing.groupby('company_id', as_index=False).agg({'payed_until':'max'})#.sort_values('payed_until', ascending=False)

In [62]:
#сразу задаю множество id компаний с действующей подпиской
active_companies_set = set(companies_by_subscription_date.query('payed_until >= @research_date')['company_id'])

In [63]:
# #проверяю количество компаний
# len(active_companies_set) + len(inactive_companies_set) == billing.company_id.nunique()

In [64]:
#задаю признак с категорией компании по ее активности
billing['is_active'] = billing.company_id.apply(lambda x: True if x in active_companies_set else False)

In [65]:
#задаю признак с категорией компании по факту оплаты
billing['is_paying'] = billing.company_id.apply(lambda x: True if x in paying_companies_set else False)

In [66]:
# сгруппированная по компаниям таблица с основными признаками компании
billing_companies = (
    billing
    .sort_values('created_at')
    .groupby('company_id', as_index=False)
    .agg(
        created_at=('created_at', 'min'),
        payed_until=('payed_until', 'max'),
        is_paying=('is_paying', 'max'),
        billing_tariff_id=('billing_tariff_id', 'last'),
        billing_tariff_price_id=('billing_tariff_price_id', 'last'),
        is_active=('is_active', 'last'),
        unique_price_ids=('billing_tariff_price_id', 'nunique')
    )
)

In [67]:
billing_companies

,company_id,created_at,payed_until,is_paying,billing_tariff_id,billing_tariff_price_id,is_active,unique_price_ids
0,1,2019-07-23 15:48:05,2021-07-25 23:59:59,True,3,12,False,4
1,5,2019-07-26 10:43:37,2025-06-19 23:59:59,True,17,25,True,13
2,6,2019-07-25 10:54:03,2019-10-25 23:59:59,False,2,6,False,1
3,11,2022-06-30 16:35:50,2023-06-30 00:00:00,False,11,38,False,2
4,13,2019-07-24 08:59:26,2020-07-31 23:59:59,False,4,16,False,2
...,...,...,...,...,...,...,...,...
1836,9976,2024-03-04 12:42:41,2025-03-04 00:00:00,False,3,54,True,1
1837,10010,2024-02-29 13:33:29,2025-03-02 23:59:59,False,7,23,True,1
1838,10077,2024-03-10 10:28:11,2025-03-11 23:59:59,False,7,23,True,1
1839,10848,2024-03-10 09:10:57,2025-03-18 23:59:59,False,2,8,True,2


Всего у нас в `billing` 1841 компания, к ним уже присвою текстовое описание признаков

In [68]:
(billing
 .groupby('company_id', as_index=False)
 .agg(unique_price_ids=('billing_tariff_price_id', 'nunique'))
)

,company_id,unique_price_ids
0,1,4
1,5,13
2,6,1
3,11,2
4,13,2
...,...,...
1836,9976,1
1837,10010,1
1838,10077,1
1839,10848,2


Далее задам категорию постоянного тарифа для компании по `billing_tariff_id` из ТЗ:
>*тарифы 4, 7, 8, 11, 19 -  “Старт”  
тарифы 2, 17 -  “Базовый”  
тарифы 9, 20 -  “Бизнес”  
тарифы 3, 10, 13, 21 -  “Профи”  
тариф 6 - “Архивный”*  

In [69]:
def tariff_category(cell):
  '''Функция категоризации тарифа по критериям ТЗ'''
  start = (4, 7, 8, 11, 19)
  base = (2, 17)
  business = (9, 20)
  pro = (3, 10, 13, 21)
  archived = 6
  if cell in start:
    return 'Старт'
  if cell in base:
    return 'Базовый'
  if cell in business:
    return 'Бизнес'
  if cell in pro:
    return 'Профи'
  if cell == archived:
    return 'Архивный'
  else:
    return 'other'

In [70]:
billing_companies['tariff'] = billing_companies.billing_tariff_id.apply(tariff_category)
billing_companies['active'] = billing_companies.is_active.map(lambda x: 'Действующая' if x == True else 'Ушедшая')
billing_companies['paying'] = billing_companies.is_paying.map(lambda x: 'Платящая' if x == True else 'Не платящая')

In [71]:
#задаю цвета для графиков
# color_discrete_map_active = {'Действующая': "#636EFA", 'Ушедшая': "#EF553B"}
# color_discrete_map_paying = {'Платящая': "#636EFA", 'Не платящая': "#EF553B"}

color_discrete_map = {
    'Действующая': "#636EFA",  #синий
    'Ушедшая': "#EF553B",      #красный
    'Платящая': "#636EFA",     #синий
    'Не платящая': "#EF553B"   #красный
    }

In [72]:
# #текстовое обозначение категории компании для графиков
# billing['is_active_str'] = billing.is_active.map(lambda x: 'Действующая' if x == True else 'Ушедшая')
# billing['is_paying_str'] = billing.is_paying.map(lambda x: 'Платящая' if x == True else 'Не платящая')

### Основная информация о клиентах

In [73]:
# @title График количества активных компаний

fig = px.pie(
    billing_companies.groupby('active', as_index=False)['company_id'].count(),
    names='active',
    values='company_id',
    color='active',
    color_discrete_map=color_discrete_map
)
fig.update_layout(title='График количества активных и неактивных компаний',
                  legend_title='Категория',
                  height=500,
                  width=800)
fig.update_traces(textinfo='label+percent',
                  hovertemplate='Категория: %{label}<br>Количество: %{value}<br>Процент: %{percent}')
fig.show()

За все время исследования 76% компаний ушло, осталась четверть

In [74]:
# @title График количества платящих и неплатящих клиентов

fig = px.pie(
    billing_companies.groupby('paying', as_index=False)['company_id'].count(),
    names='paying',
    values='company_id',
    color='paying',
    color_discrete_map=color_discrete_map
)
fig.update_layout(
    title='График количества платящих и неплатящих клиентов',
    legend_title='Категория',
    height=500,
    width=800
)
fig.update_traces(
    textinfo='label+value',
    hovertemplate='Категория: %{label}<br>Количество: %{value}<br>Процент: %{percent}'
)
fig.show()

42% из всех компаний пользовавшихся сервисом конвертируется в платящих клиентов

<!-- **Не платящие клиенты в данной задаче нам не интересны, далее будем смотреть только на платящих** -->

In [75]:
# @title График количества компаний по тарифам

fig = px.pie(
    billing_companies.groupby('tariff', as_index=False)['company_id'].count(),
    names='tariff',
    values='company_id'
)
fig.update_layout(title='График количества компаний в каждом тарифе',
                  legend_title='Тариф',
                  height=500,
                  width=800)
fig.update_traces(textinfo='label+value',
                  hovertemplate='Тариф: %{label}<br>Количество: %{value}<br>Процент: %{percent}')
fig.show()

На архивном тарифе всего 7 компаний. Большинство - 86,6% - пользуется тарифами базовый и старт

In [76]:
# @title Cводная количества уникальных тарифов для каждой компании
(billing
 .groupby('company_id', as_index=False)
 .agg(unique_tariffs=('billing_tariff_id', 'nunique'))
 .sort_values('unique_tariffs', ascending=False)
 .groupby('unique_tariffs')
 .agg(number_of_companies=('unique_tariffs','count'))
 )

,number_of_companies
unique_tariffs,
1,1601
2,183
3,41
4,10
5,3
7,3


Из 1841 компании тариф не менялся у 1601. По остальным для анализа задам последний использовавшийся тариф.

Отдельно посмотрю на компании, у которых было 2 разных тарифа, всего таких компаний 183

In [77]:
company_two_tariff_list = billing.groupby('company_id').agg({'billing_tariff_id':'nunique'}).query('billing_tariff_id == 2').index.tolist()

company_two_tariff = (
    billing
    .query('company_id in @company_two_tariff_list')
    .sort_values('created_at')
    .groupby('company_id', as_index=False)
    .agg({'billing_tariff_id':['first', 'last']})
)
company_two_tariff['tariff_first'] = company_two_tariff['billing_tariff_id', 'first'].apply(tariff_category)
company_two_tariff['tariff_last'] = company_two_tariff['billing_tariff_id', 'last'].apply(tariff_category)

company_two_tariff = (
    company_two_tariff
    .groupby('tariff_first')
    .agg({('company_id', ''):'count'})
    .merge(
        company_two_tariff
        .groupby('tariff_last')
        .agg({('company_id', ''):'count'}),
        left_index=True,
        right_index=True
    )
)
company_two_tariff.index.names = ['tariff']
company_two_tariff.columns = ['first_tariff', 'last_tariff']
company_two_tariff['diff'] = company_two_tariff.last_tariff - company_two_tariff.first_tariff

In [78]:
# @title Компании, изменявшие тариф
company_two_tariff

,first_tariff,last_tariff,diff
tariff,,,
Архивный,1,2,1
Базовый,76,79,3
Бизнес,15,18,3
Профи,10,11,1
Старт,81,73,-8


При просмотре компаний, у которых за время пользования сервисом было два разных тарифа можно сделать вывод, что аномальной миграции с какого-то определенного тарифа на другие нет.

Из 183 компаний самая большая разница в тарифе Старт - с него в совокупности перешли на другие тарифы 8 компаний.

In [79]:
# График количества компаний по тарифам
fig = px.histogram(
    billing_companies,
    x="tariff",
    color="paying",
    color_discrete_map=color_discrete_map
)
fig.update_layout(
    title='График количества компаний по тарифам и платежному статусу',
    xaxis_title='Тариф',
    yaxis_title='Количество',
    legend=dict(title='Статус компании')
)
fig.update_xaxes(categoryorder='total descending')
fig.update_traces(hovertemplate='Количество компаний: %{y}<br>Статус компании: %{fullData.name}<extra></extra><br>Тариф: %{x}')
fig.show()

Большинство пользователей на тарифах Старт и Базовый, платят на них менее половины.

**Итого по основной информации:**
- Всего компаний в датасете с платежами - 1841.
- Данные не очень качественные: в датасетах об общей информации и активностях клиентов отсутствуют данные о 1125 компаниях, среди них есть платящие.
- За все время исследования в активных клиентах остались 24% компаний.
- 42% из всех компаний пользовавшихся сервисом конвертируется в платящих клиентов.
- 87% всех клиентов пользуются тарифами **Базовый** и **Старт**.
- У 87% компаний тариф не менялся. В компаниях менявших тариф однозначной миграции с какого-либо определенного тарифа, или на какой-либо определенный, нет.

Пока работаю с датасетом `billing` посчитаю время жизни компаний: возьму за первую временную отметку первую дату создания платежа для каждой компании, и сравню ее с максимальной датой действия подписки

### Отток клиентов

In [80]:
#даты начала и конца использования сервиса каждой компанией
companies_interval = (
    billing
    .groupby('company_id', as_index=False)
    .agg({'created_at':'min', 'payed_until':'max'})
)
companies_interval['created_at'] = pd.to_datetime(companies_interval['created_at']).dt.to_period('M')
companies_interval['payed_until'] = pd.to_datetime(companies_interval['payed_until']).dt.to_period('M')

#все уникальные даты (год-месяц) за исследуемый период
date_range = pd.date_range(
    start=str(companies_interval.payed_until.min()),
    end=str(pd.to_datetime(research_date).to_period('M')),
    freq='M'
    )
year_month_series = pd.Series(date_range.to_period('M'))

#расчет количества ушедших компаний каждый месяц
cr = (
    companies_interval
    .groupby('payed_until', as_index=False)
    .agg(lost_companies=('company_id','count'))
    .merge(
        year_month_series.to_frame(),
        left_on='payed_until',
        right_on=0,
        how='right'
    )
    .drop(columns=0)
    .fillna(0)
    .sort_values('payed_until')
)
cr['lost_companies'] = cr['lost_companies'].astype(int)
cr['active_companies'] = 0

<ipython-input-80-d3625fac3fc3>:11: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [81]:
#расчет количества активных компаний каждый месяц

# Для каждого месяца проверяем, какие компании были активны
for i, row in cr.iterrows():
    # Для каждого месяца считаем активные компании
    month = row['payed_until']
    active_companies = companies_interval[
        (companies_interval['created_at'] <= month) &
        (companies_interval['payed_until'] >= month)
    ].shape[0]

    # Записываем количество активных компаний
    cr.at[i, 'active_companies'] = active_companies

In [82]:
#расчет метрики churn_rate
cr['churn_rate'] = round(cr.lost_companies/cr.active_companies, 3)
cr['payed_until'] = cr['payed_until'].astype('str') #переводим дату в строку для графика
cr.head()

,payed_until,lost_companies,active_companies,churn_rate
0,2019-10,2,25,0.080
1,2019-11,3,26,0.115
2,2019-12,5,30,0.167
3,2020-01,3,34,0.088
4,2020-02,0,34,0.000


In [83]:
cr.head()

,payed_until,lost_companies,active_companies,churn_rate
0,2019-10,2,25,0.080
1,2019-11,3,26,0.115
2,2019-12,5,30,0.167
3,2020-01,3,34,0.088
4,2020-02,0,34,0.000


In [84]:
# @title Churn Rate и количество компаний по месяцам

# Создаем пустой subplot с нужной конфигурацией (2 ряда, 2 колонки, где вторая строка объединяет 2 колонки)
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "xy"}, {"type": "xy"}], [{"colspan": 2}, None]],
    subplot_titles=("График количества активных компаний", "График количества ушедших компаний", "Churn Rate")
)

# График 1: Количество активных компаний
fig1 = px.line(cr, y='active_companies', x='payed_until')
fig.add_trace(
    fig1['data'][0].update(hovertemplate='Дата: %{x}<br>Активные компании: %{y}'),
    row=1, col=1
)

# График 2: Количество ушедших компаний
fig2 = px.line(cr, y='lost_companies', x='payed_until')
fig.add_trace(
    fig2['data'][0].update(hovertemplate='Дата: %{x}<br>Ушедшие компании: %{y}'),
    row=1, col=2
)

# График 3: Churn Rate (на всю ширину второго ряда)
fig3 = px.line(cr, y='churn_rate', x='payed_until')
fig.add_trace(
    fig3['data'][0].update(hovertemplate='Дата: %{x}<br>Churn rate: %{y:.1%}'),
    row=2, col=1
)

# Настраиваем оси и метки для первого графика
fig.update_yaxes(title_text='Количество', row=1, col=1)
# Настраиваем оси и метки для второго графика
fig.update_yaxes(title_text='Количество', row=1, col=2)
# Настраиваем оси и метки для третьего графика
fig.update_yaxes(title_text='Churn rate, %', tickformat='.0%', row=2, col=1)

# Общие настройки
fig.update_layout(
    title_text="Объединенные графики",
    xaxis_title='Дата',
    showlegend=False,
    height=800,  # Высота всего графика
)

# Показываем график
fig.show()

Пика количества клиентов сервис достиг в начале 2022 года, сильно увеличив их число за предыдущий год.

Активный отток клиентов начался в начале 2023 и составляет порядка 10% ежемесячно. В абсолютных числах это больше 50 клиентов каждый месяц.

In [85]:
# @title График доли ушедших компаний по тарифам

# сводная количества компаний и доли ушедших в каждом тарифе
lr = round(
    billing_companies
    .query('is_active == False')
    .groupby('tariff')
    .agg(lost_ratio=('company_id','count'))
    /
    billing_companies
    .groupby('tariff')
    .agg(lost_ratio=('company_id','count')),
    2
)
pv = (
    billing_companies
    .groupby('tariff')
    .agg(companies=('company_id','count'))
    .merge(
        lr,
        left_index=True,
        right_index=True
    )
    .sort_values('companies', ascending=False)
)

# график
fig = px.bar(
    pv,
    x=pv.index,
    y='lost_ratio',
    opacity=0.7
)
fig.update_layout(
    title='Процент ушедших компаний в каждом тарифе',
    xaxis_title='Тариф',
    yaxis_title='Ушло',
    yaxis_tickformat = '.0%'
)
colors = ['red']*2 + ['blue']*3
fig.update_traces(
    marker=dict(color=colors),
    #text=pv['companies'].apply(lambda x: f'Компаний: {x}'),
    customdata=pv[['companies']],
    hovertemplate='Тариф: %{label}<br>Доля ушедших: %{value}<br>Количество компаний: %{customdata[0]}'
)
fig.update_yaxes(range=[0, 1])

fig.show()

Меньше всего ушли с тарифа бизнес - 64%, но и компаний на нем не так много. С основных тарифов Базовый и Старт ушли 76 и 79 процентов клиентов соответственно

In [86]:
lost_by_tariff = (
    billing
    .query('is_active == False')
    .sort_values('created_at')
    .groupby(['company_id'], as_index=False)
    .agg({'payed_until':'max', 'billing_tariff_id':'last'})
)
lost_by_tariff['tariff'] = lost_by_tariff.billing_tariff_id.apply(tariff_category)
lost_by_tariff['payed_until'] = pd.to_datetime(lost_by_tariff['payed_until']).dt.to_period('M')

lost_by_tariff = (
    lost_by_tariff
    .groupby(['payed_until', 'tariff'], as_index=False)
    .agg(lost_companies=('company_id','count'))
)
lost_by_tariff['payed_until'] = lost_by_tariff['payed_until'].astype('str')

In [87]:
# @title График количества ушедших компаний с тарифов по месяцам

fig = px.line(lost_by_tariff,
             y='lost_companies',
             x='payed_until',
              color='tariff'
             )
fig.update_layout(
    title='График количества ушедших компаний с тарифов по месяцам',
    yaxis_title='Количество',
    xaxis_title='Дата',
    legend_title=('Тариф')
)
fig.update_traces(hovertemplate='Дата: %{x}<br>Компаний: %{y}<extra></extra>')
fig.show()

В 2023 году, когда начался отток клиентов, больше всех уходят с тарифа Базовый. Пик оттока с этого тарифа был летом - за два месяца ушло больше 100 компаний.

С тарифа Старт уходят чуть реже, но тенденция повышенного оттока клиентов в 2023 году наблюдается и у него.

In [88]:
# @title Старт: Количество ушедших компаний тарифа по ID оплаты

pv = (
    billing
    .query('is_active == False')
    .sort_values('created_at')
    .groupby(['company_id'], as_index=False)
    .agg({'payed_until':'max', 'billing_tariff_id':'last', 'billing_tariff_price_id':'last'})
)
pv['tariff'] = pv.billing_tariff_id.apply(tariff_category)
pv = (
    pv
    .query('tariff == "Старт"')
    .groupby(['billing_tariff_price_id'], as_index=False)
    ['company_id'].count()
)
# график
fig = px.bar(
    pv.sort_values('company_id', ascending=False),
    x='billing_tariff_price_id',
    y='company_id',
    opacity=0.7
)
fig.update_layout(
    title='Количество ушедших компаний тарифа Старт по ID оплаты',
    xaxis_title='Тариф',
    yaxis_title='Количество'
)
colors = ['red'] + ['blue'] * (pv.billing_tariff_price_id.nunique() - 1)
fig.update_traces(
    marker=dict(color=colors),
    #text=pv['company_id'],
    hovertemplate='ID оплаты: %{label}<br>Количество: %{value}'
)
#fig.update_xaxes(categoryorder='total descending')
fig.show()

На тарифе Старт больше всего клиентов уходят с ID оплаты 23

In [89]:
# @title Базовый: Количество ушедших компаний тарифа по ID оплаты

pv = (
    billing
    .query('is_active == False')
    .sort_values('created_at')
    .groupby(['company_id'], as_index=False)
    .agg({'payed_until':'max', 'billing_tariff_id':'last', 'billing_tariff_price_id':'last'})
)
pv['tariff'] = pv.billing_tariff_id.apply(tariff_category)
pv = (
    pv
    .query('tariff == "Базовый"')
    .groupby(['billing_tariff_price_id'], as_index=False)
    ['company_id'].count()
)
# график
fig = px.bar(
    pv.sort_values('company_id', ascending=False),
    x='billing_tariff_price_id',
    y='company_id',
    opacity=0.7
)
fig.update_layout(
    title='Количество ушедших компаний тарифа Базовый по ID оплаты',
    xaxis_title='Тариф',
    yaxis_title='Количество'
)
colors = ['red']*2 + ['blue'] * (pv.billing_tariff_price_id.nunique() - 2)
fig.update_traces(
    marker=dict(color=colors),
    #text=pv['company_id'],
    hovertemplate='ID оплаты: %{label}<br>Количество: %{value}'
)
#fig.update_xaxes(categoryorder='total descending')
fig.show()

На тарифе Базовый больше всего клиентов уходят с ID оплаты 24 и 8

In [90]:
billing_companies.groupby('unique_price_ids').agg({'company_id':'count', 'is_active':'sum'})

,company_id,is_active
unique_price_ids,,
1,1428,287
2,266,90
3,90,38
4,19,8
5,10,4
6,12,5
7,5,3
8,6,3
11,1,0


In [91]:
# @title График количества ушедших компаний по количеству разных ID оплаты

fig = px.histogram(
    billing_companies,
    x="unique_price_ids",
    color='active',
    barmode='group',
    color_discrete_map=color_discrete_map
)
fig.update_layout(
    title='График количества ушедших компаний по количеству разных ID оплаты',
    xaxis_title='Количество ID оплаты',
    yaxis_title='Количество',
    legend=dict(title='Статус компании'),
)
fig.update_traces(
    texttemplate='%{y}',
    textposition='outside',
    hovertemplate='Количество компаний: %{y}<br>Статус: %{fullData.name}<extra></extra><br>Количество ID оплаты: %{x}')
fig.update_xaxes(range=[0.5, 5.5])
fig.update_yaxes(showticklabels=False)

fig.show()

У большинства ушедших компаний не менялся ID оплаты

**Итого по оттоку:**
- Активный отток клиентов начался в начале 2023 и составляет порядка 10% клиентов ежемесячно.
- За все время меньше всего ушли с тарифа бизнес - 64% клиентов. С основных тарифов **Базовый** и **Старт** ушли 76 и 79 процентов клиентов соответственно.
- С 2023 года по настоящее время больше всех уходят с тарифа **Базовый**. С тарифа **Старт** уходят меньше компаний, но тенденция повышенного оттока клиентов в 2023 году наблюдается и у него.
- На тарифе **Старт** больше всего клиентов уходят с ID оплаты `23`
- На тарифе **Базовый** больше всего клиентов уходят с ID оплаты `24` и `8`
- У 62% ушедших компаний не менялся ID оплаты

### Lifetime

In [92]:
companies_lifetime = billing_companies[['company_id', 'payed_until', 'created_at', 'billing_tariff_price_id', 'tariff', 'active']]

In [93]:
#создаю поле с временем подписки компании
companies_lifetime = companies_lifetime.copy()
companies_lifetime['subscription_period'] = companies_lifetime.payed_until - companies_lifetime.created_at

In [94]:
#оставляю количество месяцев
companies_lifetime['subscription_period_months'] = companies_lifetime.subscription_period.dt.days // 30

In [95]:
#минимальное количество месяцев подписки компании
companies_lifetime.subscription_period_months.min()

-1

In [96]:
#убираю отрицательные значения
companies_lifetime['subscription_period_months'] = companies_lifetime.subscription_period_months.apply(lambda x: 0 if x < 0 else x)

In [97]:
# companies_lifetime['tariff_category'] = companies_lifetime.billing_tariff_id.apply(tariff_category)
# companies_lifetime['is_active_str'] = companies_lifetime['is_active'].map(lambda x: 'Действующая' if x == True else 'Ушедшая')

In [98]:
# @title Диаграмма размаха лайфтайма компаний
fig = px.box(companies_lifetime,
             y="subscription_period_months")
fig.update_layout(height=700,
                  width=1000,
                  title='График длительности подписки компаний',
                  yaxis_title = 'Количество месяцев')

fig.show()

Выбросы есть, медианое время жизни всех компаний - 18 месяцев

In [99]:
# @title Диаграмма размаха лайфтайма компаний по статусам
fig = px.box(
    companies_lifetime,
    x='active',
    y='subscription_period_months',
    category_orders={'active': companies_lifetime.active}
)
fig.update_layout(
    height=700,
    width=1000,
    title='График длительности подписки ушедших и действующих компаний',
    xaxis_title='Статус компании',
    yaxis_title='Количество месяцев'
)
fig.update_yaxes(range=[0, 80])
fig.show()

Медианное значение периода подписки у активных компаний 20 месяцев. Период подписки у ушедших компаний меньше - 14 месяцев. Высокие значения выбросов встречаются у активных компаний - значит кто-то оформляет подписку на несколько лет вперед.

In [100]:
#чищу companies_lifetime от выбросов
subscription_period_months_upper_bound = companies_lifetime.subscription_period_months.quantile(0.75) + 1.5 * iqr(companies_lifetime.subscription_period_months)
companies_lifetime = companies_lifetime.query('subscription_period_months <= @subscription_period_months_upper_bound')

In [101]:
# @title График количества компаний по времени использования сервиса
fig = px.histogram(
    companies_lifetime,
    x='subscription_period_months',
    color='active',
    color_discrete_map=color_discrete_map,
    nbins=companies_lifetime.subscription_period_months.nunique()
)
fig.update_layout(
    title='График количества компаний-клиентов по времени использования сервиса',
    xaxis_title='Количество месяцев',
    yaxis_title='Количество компаний',
    legend=dict(title='Статус компании')
)
fig.update_traces(
    customdata=companies_lifetime.active,
    hovertemplate='Компаний: %{y}<br>Месяцев: %{x}<br>Статус компании: %{fullData.name}<extra></extra>'
)
fig.show()

Большинство компаний пользуется сервисом 12, 18 или 24 месяца, после этого же преиода многие уходят.

In [102]:
f"Доля ушедших компаний до года - {companies_lifetime[(companies_lifetime.active == 'Ушедшая') & (companies_lifetime.subscription_period_months < 12)].shape[0] / billing.company_id.nunique():.0%}"

'Доля ушедших компаний до года - 18%'

Почти 20% компаний уходят до года

In [103]:
# @title Пересечение id тарифов оплаты
(
    companies_lifetime
    .pivot_table(index='tariff',
                 columns='billing_tariff_price_id',
                 values='company_id',
                 aggfunc='count')
    .fillna(0)
    .astype('int')
    .T
    #.style.background_gradient(axis=0, cmap='PuBuGn', low=0.7, high=1.5)
)

tariff,Архивный,Базовый,Бизнес,Профи,Старт
billing_tariff_price_id,,,,,
1,0,6,0,0,0
10,0,0,0,3,0
11,0,0,0,1,0
12,0,0,0,9,0
13,0,0,0,0,12
14,0,0,0,0,24
15,0,1,0,0,1
16,0,0,0,0,25
17,1,0,0,0,0


id тарифов оплаты в тарифах пересекаются только в единичных случаях. Почти для всех тарифов id оплаты уникальные.

In [104]:
# @title График рассеяния даты привлечения компании к лайфтайму
fig = px.scatter(
    companies_lifetime,
    x="created_at",
    y="subscription_period_months",
    color="active",
    color_discrete_map=color_discrete_map,
    opacity=0.5
)
fig.update_layout(
    title='График рассеяния даты привлечения компании-клиента к времени жизни',
    xaxis_title='Дата привлечения',
    yaxis_title='Время жизни, мес.',
    legend=dict(title='Статус компании')
)
fig.update_traces(hovertemplate='Привлечение: %{x}<br>Лайфтайм: %{y} мес.<br>Статус компании: %{fullData.name}<extra></extra>')
fig.show()

Из графика видно, что много компаний было привлечено в 21-22 году. Большинство из них пользовались сервисом 12, 18 и 24 месяца, затем ушли.

In [105]:
tariff_mean_lifetime = companies_lifetime.groupby('tariff').agg({'subscription_period_months':'mean', 'company_id':'count'}).round(2).sort_values('company_id')#, ascending=False)
tariff_mean_lifetime

,subscription_period_months,company_id
tariff,,
Архивный,21.43,7
Профи,16.98,80
Бизнес,17.66,154
Старт,14.98,709
Базовый,16.25,856


In [106]:
# @title График среднего лайфтайма компаний по тарифам
fig = px.bar(
    tariff_mean_lifetime,
    y=tariff_mean_lifetime.index,
    x='subscription_period_months',
    opacity=0.75
)
fig.update_layout(
    title='График среднего количества месяцев жизни по тарифам',
    yaxis_title='Тариф',
    xaxis_title='Количество месяцев',
    height=500,
    width=800
)
colors = ['blue']*3 + ['red']*2
fig.update_traces(
    marker=dict(color=colors),
    text=tariff_mean_lifetime['company_id'].apply(lambda x: f'Компаний: {x}'),
    customdata=tariff_mean_lifetime[['company_id']],
    hovertemplate='Тариф: %{label}<br>Средний лайфтайм: %{value}<br>Количество компаний: %{customdata[0]}'
)
fig.show()

Различия в среднем времени подписки компаний по тарифам, за исключением архивного, в котором всего 6 компаний, небольшие, в пределах трех месяцев

Ни в одном тарифе (опять же за исключением архивного) средний лайфтайм не превышает 19 месяцев.

In [107]:
#количество компаний, всегда платящих по одному коэфиценту
len(billing.groupby('company_id').agg({'billing_tariff_price_id':'nunique'})) - \
len(billing.groupby('company_id').agg({'billing_tariff_price_id':'nunique'}).query('billing_tariff_price_id > 1'))

1428

У 1428 компаний не изменялся коэфицент оплаты тарифа

#### Графики среднего лайфтайма компании внутри тарифов по id коэфицента оплаты

In [108]:
def bill_single_coef_by_tariff(tariff):
  '''Функция создания графика среднего количества месяцев жизни id коэфицента оплаты для выбранного тарифа у компаний, всегда платящих по одномцу коэфиценту'''

  #айдишники компаний, всегда платящих по одному коэфиценту
  companies_single_billing_tariff_price_id_array = billing.groupby('company_id').agg({'billing_tariff_price_id':'nunique'}).query('billing_tariff_price_id == 1').index.values

  #сводная по компаниям на выбранном тарифе платящим по одному коэфиценту
  pv = (
      companies_lifetime
      .query('tariff == @tariff and company_id in @companies_single_billing_tariff_price_id_array')
      .groupby('billing_tariff_price_id', as_index=False)
      .agg({'subscription_period_months':'mean', 'company_id':'count'})
      .round(2)
      .sort_values('company_id')
      )
  #график
  fig = px.bar(
      pv,
      y='billing_tariff_price_id',
      x='subscription_period_months',
      opacity=0.7
  )
  fig.update_layout(
      title=f'График среднего лайфтайма компании в тарифе {tariff} по id коэфицента оплаты',
      yaxis_title=f'ID коэфицента оплаты тарифа {tariff}',
      xaxis_title='Средний лайфтайм (мес.)',
      height=600,
      width=1200
  )
  colors = ['blue'] * (pv['billing_tariff_price_id'].nunique() - 3) + ['red', 'red', 'red']

  fig.update_traces(
      marker=dict(color=colors),
      text=pv['company_id'].apply(lambda x: f'Компаний: {x}'),
      customdata=pv[['company_id']],
      hovertemplate='ID оплаты: %{label}<br>Средний лайфтайм: %{value}<br>Количество компаний: %{customdata[0]}'
  )
  #fig.update_xaxes(categoryorder='total descending')
  #fig.show()
  return fig

In [109]:
# @title Старт
bill_single_coef_by_tariff('Старт').show()

На втором по популярности тарифе Старт почти половина всех пользователей этого тарифа платит по id оплаты `23`, у которого средний лайфтайм 12 месяцев.

При этом у нас есть id `34`, в котором средний лайфтайм в два раза больше - 23 месяца, и в нем достаточно много пользователей.

In [110]:
# @title Базовый
bill_single_coef_by_tariff('Базовый').show()

На самом популярном тарифе Базовый больше всего пользователей платит по id оплаты `24` у которого средний лайфтайм 17 с половиной месяцев. Но почти столько эе пользователей платит по id `8`, у которого лайфтайм 11 месяцев.

И у нас есть id `35`, в котором средний лайфтайм почти 23 месяца, и в нем тоже достаточно много пользователей.

In [111]:
# @title Бизнес
bill_single_coef_by_tariff('Бизнес').show()

На id оплаты `36` тарифа Бизнес компании пользуются сервисом дольше всего.

На id оплаты `30` больше всего пользователей, но средний лайфтайм у них на шесть месяцев ниже, чем у тех, кто платит по id `36`

Также, на id оплаты `19` средний лайфтайм 29 месяцев, но по этому id платит только одна компания, поэтому это значение может непоказательно

In [112]:
# @title Профи
bill_single_coef_by_tariff('Профи').show()

Внутри тарифа Профи больше всего компаний платят по id `37`, средний лайфтайм на этом id тоже выше, чем в других.  

Есть id с лайфтаймами по 24 и 25 месяцев, но только с одной компанией, платящей по этому id, что непоказательно.

**Итого по лайфтайму:**
- Медианое время жизни всех компаний - 18 месяцев
- Медианное вермя жизни у активных компаний 20 месяцев, у ушедших компаний 14 месяцев.
- Большинство компаний пользуется сервисом 12, 18 или 24 месяца, после этого же преиода многие уходят.
- Доля ушедших компаний до года - 18%
- Различия в среднем времени подписки компаний по тарифам, за исключением архивного, в котором всего 6 компаний, небольшие, в пределах трех месяцев
- На тарифе **Старт** почти половина всех пользователей этого тарифа платит по ID оплаты `23`, у которого средний лайфтайм 12 месяцев. В этом же тарифе есть ID `34`, в котором средний лайфтайм в два раза больше - 23 месяца.
- На тарифе **Базовый** больше всего пользователей платит по ID оплаты `24` и `8` на которых лайфтайм 17 и 11 месяцев. В этом же тарифе есть ID `35`, в котором средний лайфтайм 23 месяца.

### **Вывод по данным о платежах**

**Основная информация:**
- Всего компаний в датасете с платежами - 1841.
- Данные не очень качественные: в датасетах об общей информации и активностях клиентов отсутствуют данные о 1125 компаниях, среди них есть платящие.
- За все время исследования в активных клиентах остались 24% компаний.
- 42% из всех компаний пользовавшихся сервисом конвертируется в платящих клиентов.
- 87% всех клиентов пользуются тарифами **Базовый** и **Старт**.
- У 87% компаний тариф не менялся. В компаниях менявших тариф однозначной миграции с какого-либо определенного тарифа, или на какой-либо определенный, нет.

**Отток клиентов:**
- Активный отток клиентов начался в начале 2023 и составляет порядка 10% клиентов ежемесячно.
- За все время меньше всего ушли с тарифа бизнес - 64% клиентов. С основных тарифов **Базовый** и **Старт** ушли 76 и 79 процентов клиентов соответственно.
- С 2023 года по настоящее время больше всех уходят с тарифа **Базовый**. С тарифа **Старт** уходят меньше компаний, но тенденция повышенного оттока клиентов в 2023 году наблюдается и у него.
- На тарифе **Старт** больше всего клиентов уходят с ID оплаты `23`
- На тарифе **Базовый** больше всего клиентов уходят с ID оплаты `24` и `8`

**Лайфтайм клиентов:**
- Медианое время жизни всех компаний - 18 месяцев
- Медианное вермя жизни у активных компаний 20 месяцев, у ушедших компаний 14 месяцев.
- Большинство компаний пользуется сервисом 12, 18 или 24 месяца, после этого же преиода многие уходят.
- Доля ушедших компаний до года - 18%
- Различия в среднем времени подписки компаний по тарифам, за исключением архивного, в котором всего 6 компаний, небольшие, в пределах трех месяцев
- На тарифе **Старт** почти половина всех пользователей этого тарифа платит по id оплаты `23`, у которого средний лайфтайм 12 месяцев. В этом же тарифе есть id `34`, в котором средний лайфтайм в два раза больше - 23 месяца.
- На тарифе **Базовый** больше всего пользователей платит по id оплаты `24` и `8` на которых лайфтайм 17 и 11 месяцев. В этом же тарифе есть id `35`, в котором средний лайфтайм 23 месяца.


Дальше надо добавлять в работу другие датасеты, количество исследуемых компаний уменьшится до 716

## Данные о компаниях

In [113]:
#смотрю уникальные значения working_direction для категоризации
#companies.working_direction.unique()

In [114]:
def direction_category(row):
    if 'строитель' in row['working_direction'].lower():
        if 'ремонт' in row['working_direction'].lower() or 'отдел' in row['working_direction'].lower():
            return 'строительство и ремонт'
        return 'строительство'
    if 'ремонт' in row['working_direction'].lower() or 'отдел' in row['working_direction'].lower():
        return 'ремонт'
    if 'инженер' in row['working_direction'].lower():
      return 'инженерные системы'
    if 'электро' in row['working_direction'].lower():
      return 'электромонтаж'
    if row['working_direction'] == '-':
      return 'no information'
    return 'other'

In [115]:
companies['direction_category'] = companies.apply(direction_category, axis=1)

In [116]:
round(companies.groupby('direction_category')['id'].count() / len(companies), 2)

,id
direction_category,
no information,0.64
other,0.05
инженерные системы,0.01
ремонт,0.05
строительство,0.22
строительство и ремонт,0.03
электромонтаж,0.01


По 64% компаний нет информации о сфере деятельности

In [117]:
companies = companies.rename(columns={'id': 'company_id'}).merge(billing.groupby('company_id', as_index=False).agg({'is_active':'max'})[['company_id', 'is_active']], on='company_id', how='left')

In [118]:
companies.is_active.isna().sum() / len(companies)

0.0

Все компании есть в  `billing`, категория присвоилась всем

In [119]:
#категоризирую количество сотрудников
def employer_count(x):
  if x == 1:
    return '1-4'
  if x == 2:
    return '5-10'
  if x == 3:
    return '11-20'
  if x == 4:
    return '21-30'
  if x == 5:
    return '31-50'
  if x == 6:
    return '51-100'
  if x == 7:
    return '101-200'
  if x == 8:
    return '>201'
  return '-'

In [120]:
companies = companies.assign(employer_count=lambda x: x['employer_count_interval'].apply(employer_count))

In [121]:
#срез по активным компаниям
active_companies = companies.query('is_active == True')

In [122]:
# @title График количества компаний по категориям сфер деятельности
pv = active_companies.groupby('direction_category', as_index=False)['company_id'].count().sort_values('company_id', ascending=False)
fig = px.bar(
    pv,
    x='direction_category',
    y='company_id',
    opacity=0.7
)
fig.update_layout(
    title='График количества компаний-клиентов по категориям сфер деятельности',
    xaxis_title='Категория',
    yaxis_title='Количество компаний'
)
colors = ['red'] + ['blue']*len(pv)
fig.update_traces(
    marker=dict(color=colors),
    hovertemplate='Категория: %{x}<br>Компаний: %{y}'
)
fig.update_xaxes(categoryorder='total descending')
fig.show()

По большей части компаний информации о сфере деятельности нет, но из оставшихся - большинство строительные.

In [123]:
# @title График количества компаний по количеству сотрудников
pv = active_companies.groupby('employer_count', as_index=False)['company_id'].count().sort_values('company_id', ascending=False)
fig = px.bar(
    pv,
    x='employer_count',
    y='company_id',
    opacity=0.7
)
fig.update_layout(
    title='График количества компаний-клиентов по количеству сотрудников',
    xaxis_title='Количество сотрудников',
    yaxis_title='Количество компаний'
)
colors = ['red']*2 + ['blue']*len(pv)
fig.update_traces(
    marker=dict(color=colors),
    hovertemplate='Сотрудников: %{x}<br>Компаний: %{y}'
)
fig.update_xaxes(categoryorder='total descending')
fig.show()

Большинство активных компаний - не большие, до 10 человек

### Портрет компании-пользователя

По большинству компаний нет информации о сфере деятельности. По тем что имеется: основные сферы - строительство и ремонт. Количество сотрудников редко более 10 человек.


## Данные об активностях

In [124]:
#группирую по компаниям, беру количество месяцев активности каждой, присоединяю признак ушла/не ушла
companies_by_number_active_months = (
    activities
    .groupby('company_id', as_index=False)
    .agg({'month':'count'})
    .merge(companies_lifetime[['company_id','active']],
           on='company_id',
           how='left')
    )

In [125]:
# @title График продолжительности использования сервиса компаний

fig = px.box(companies_by_number_active_months,
             x=companies_by_number_active_months.active,
             y='month',
             category_orders={'active': companies_by_number_active_months.active}
            )
fig.update_layout(height=700,
                  width=1000,
                  title='График длительности подписки ушедших и действующих компаний',
                  xaxis_title='Статус компании',
                  yaxis_title='Количество месяцев'
                 )
fig.update_yaxes(range=[0, 40])
fig.show()

Данный график подтверждает вывод по активности компаний, сделанный по датасету `billing`. Активные компани-клиенты чаще пользуются сервисом.

In [126]:
#сводная для графиков активностей компаний
d_count = dict.fromkeys([activities.columns[1]], 'count')
d_mean = dict.fromkeys(activities.columns[2:], 'mean')
companies_by_mean_activity = (
    activities
    .groupby('company_id')
    .agg({**d_count, **d_mean})
    .round(2)
    .merge(companies_lifetime[['company_id','active', 'subscription_period_months']],
           on='company_id',
           how='left'
           )
    )
# #текстовое обозначение категории компании для графиков
# companies_by_mean_activity['is_active_str'] = companies_by_mean_activity.is_active.apply(lambda x: 'Действующая' if x == True else 'Ушедшая')

In [127]:
def plot_companies_activity(col, sign, scatter_yaxes_max=None, box_yaxes_max=None):
    '''Функция создания двух графиков

    График рассеяния среднего количества признака компании к времени жизни,
    Диаграмма размаха количества признака компаний с разделением по статусу'''

    scatter_yaxes_range = [0, scatter_yaxes_max]
    box_yaxes_range = [0, box_yaxes_max]

    #создаем сетку для графиков
    fig = make_subplots(
        rows=2,
        cols=1,
        subplot_titles=(
            f'График рассеяния среднего количества {sign} компании к времени жизни',
            f'График среднего количества {sign} ушедших и действующих компаний'
        )
    )

    #добавляем первый график
    fig.add_trace(
        go.Scatter(
            x=companies_by_mean_activity.subscription_period_months,
            y=col,
            mode='markers',
            marker=dict(color=companies_by_mean_activity.active.map(color_discrete_map),
                        opacity=0.5
            ),
            showlegend=False,
            hoverinfo='text',
            hovertext=[
                f'Статус компании: {active}<br>Лайфтайм: {sub_period}<br>{sign.capitalize()}: {c}<extra></extra>'
                for active, sub_period, c in zip(
                    companies_by_mean_activity.active,
                    companies_by_mean_activity.subscription_period_months,
                    col
                )
            ],
            hovertemplate='%{hovertext}'
        ),
        row=1,
        col=1
    )

    #добавляем второй график
    fig.add_trace(
        go.Box(
            x=companies_by_mean_activity.active,
            y=col,
            marker=dict(color='#636EFA'),
            showlegend=False,
        ),
        row=2,
        col=1
    )

    # Обновляем макет графиков
    fig.update_layout(
        height=950,
        width=1300,
        title=f'Графики среднего количества {sign} компаний'
    )

    #устанавливаем ограничения осей и названия для первого графика
    fig.update_xaxes(
        range=[0, 60],
        title='Количество месяцев',
        row=1,
        col=1
    )
    fig.update_yaxes(
        range=scatter_yaxes_range,
        title=f'Среднее количество {sign}',
        row=1,
        col=1
    )

    #устанавливаем ограничения осей для второго графика
    fig.update_xaxes(
        title='Статус компании',
        row=2,
        col=1
    )
    fig.update_yaxes(
        range=box_yaxes_range,
        title=f'Среднее количество {sign}',
        row=2,
        col=1
    )

    #отображаем графики
    pio.show(fig)

In [128]:
# @title График рассеяния среднего количества счетов компании к времени жизни

plot_companies_activity(companies_by_mean_activity.billing, 'счетов', 2.5, 2)

In [129]:
#корреляция времени жизни к среднему исследуемого признака
companies_by_mean_activity.subscription_period_months.corr(companies_by_mean_activity.billing)

-0.14152400900903772

Корреляции времени жизни компании к среднему количеству ее счетов почти нет.

Ушедшими компаниями в среднем выставляется больше счетов, чем действующими

In [130]:
# @title График рассеяния среднего количества складской активности компании к времени жизни

plot_companies_activity(companies_by_mean_activity.stocks, 'складской активности', 40, 20)

In [131]:
companies_by_mean_activity.subscription_period_months.corr(companies_by_mean_activity.stocks)

0.15099278405679348

У действующих компаний больше среднее значение складской активности, чем у ушедших.

Корреляции времени жизни компании к среднему количеству ее складской активности, также, почти нет.

In [132]:
# @title График рассеяния среднего количества бекапов компании к времени жизни

plot_companies_activity(companies_by_mean_activity.backups, 'бекапов', 10, 5)

In [133]:
companies_by_mean_activity.subscription_period_months.corr(companies_by_mean_activity.backups)

0.11356896097674861

Ушедшие компании в среднем почти не делают бекапов, фактически, все значения - выбросы. У действующих компаний ситуация немногим лучше, но медиана так же 0.

Корреляции времени жизни компании к среднему количеству ее бекапов, также, почти нет.

In [134]:
# @title График рассеяния среднего количества проектов компании к времени жизни

plot_companies_activity(companies_by_mean_activity.projects, 'проектов', 40, 20)

In [135]:
companies_by_mean_activity.subscription_period_months.corr(companies_by_mean_activity.projects)

0.1567948320204242

У действующих компаний в среднем немного больше проектов, медианное значение 3.6. У ушедших - на 1 меньше - 2.6

Корреляции времени жизни компании к среднему количеству ее проектов, также, почти нет.

In [136]:
# @title График рассеяния среднего количества контрагентов компании к времени жизни

plot_companies_activity(companies_by_mean_activity.contractors, 'контрагентов', 40, 40)

In [137]:
companies_by_mean_activity.subscription_period_months.corr(companies_by_mean_activity.contractors)

0.052231698834098526

У действующих компаний в среднем на 2 контрагента больше.

Корреляции времени жизни компании к среднему количеству ее контрагентов нет.

In [138]:
# @title График рассеяния среднего количества заказов материалов компании к времени жизни

plot_companies_activity(companies_by_mean_activity.resource_orders, 'заказов материалов', 50, 40)

In [139]:
companies_by_mean_activity.subscription_period_months.corr(companies_by_mean_activity.resource_orders)

0.11703173458234926

Действующие компании в 3 раза больше заказывают материалов, чем ушедшие.

Корреляции времени жизни компании к среднему количеству ее проектов, также, почти нет.

#### **Вывод по исследованию активностей компаний**

Корреляции средней активнойсти компаний к их времени жизни отсутствуют, т.е. долго пользуются сервисом как активные компании-клиенты, так и уже ушедшие, и наоборот.

Все компании в среднем выставляют мало счетов: ушедшие 0.5 в месяц, действующие - 0.33. И почти все не делают бекапы - медианеное среднее значение - 0 у обеих групп.  

Но у действующих компаний больше среднемесячное значение складской активности (3.7), контрагентов (3.6) и заказов материалов (3), чем у ушедших - 2.7; 2.7; 1 соотвестсвенно.

# Проверка гипотез

In [140]:
#уровень статзначимости
alpha = .05

## Группы : Ушедшие и неушедшие компании

In [141]:
def equal_var(
    group_a: pd.core.series.Series,
    group_b: pd.core.series.Series
    ) -> None:
  '''
  Проверка на соотношение дисперсий групп
  Если дисперсии групп равны, то проводим t-тест Стьюдента
  Если дисперсии различаются более чем в 4 раза, значит они не равны, проводим t-тест Уэлча
  Args:
    group_a (pd.core.series.Series): поле датафрейма с наблюдениями группы A
    group_b (pd.core.series.Series): поле датафрейма с наблюдениями группы B
  Returns:
    None: Используемый тест выводится функцией print
  '''
  var_a = np.var(group_a)
  var_b = np.var(group_b)
  if var_a / var_b > 4 or var_a / var_b < 0.25:
    print('Welch’s t-test')
  else:
    print('Student’s t-test')

### Среднее количество активностей

Формулируем гипотезы:

>$H_0$ - Ушедшие и неушедшие компании равны по количеству активностей
>
>$H_1$ - Ушедшие и неушедшие компании не равны по количеству активностей
>

In [142]:
companies_by_mean_activity['mean_number_of_activities'] = companies_by_mean_activity[['billing', 'stocks', 'backups', 'projects', 'contractors', 'resource_orders']].mean(axis=1)

In [143]:
group_a = companies_by_mean_activity.query('active == "Действующая"')['mean_number_of_activities']
group_b = companies_by_mean_activity.query('active == "Ушедшая"')['mean_number_of_activities']

In [144]:
equal_var(group_a, group_b)

Student’s t-test


In [145]:
# T-тест между двумя группами
t_stat, p_val = ttest_ind(group_a, group_b, equal_var=True)

print(f"Значение t-статистики: {t_stat}")
print(f"P-value для t-теста: {p_val}")
print('------------')

if p_val > alpha:
  print('Нулевая гипотеза верна')
else:
  print('Отклоняем нулевую гипотезу')

Значение t-статистики: 5.513701840800724
P-value для t-теста: 4.96437926422585e-08
------------
Отклоняем нулевую гипотезу


P-value очень мало, отклоняем нулевую гипотезу о том, что ушедшие и неушедшие компании равны по количеству активностей

### Численность сотрудников

Формулируем гипотезы:

>$H_0$ - Ушедшие и неушедшие компании равны по численности сотрудников
>
>$H_1$ - Ушедшие и неушедшие компании не равны по численности сотрудников
>

In [146]:
companies['is_employers_less_11'] = companies.employer_count.apply(lambda x: 1 if x == '1-4' or x == '5-10' else 0)

In [147]:
pv = pd.crosstab(companies['is_employers_less_11'], companies['is_active'])
pv

is_active,False,True
is_employers_less_11,,
0,44,34
1,420,218


In [148]:
# Тест хи-квадрат
chi2, p_val, dof, expected = chi2_contingency(pv)

In [149]:
# Выведем результаты теста
print(f"Chi-squared: {chi2}")
print(f"P-value: {p_val}")
print(f"Degrees of freedom: {dof}")
print("Expected frequencies:")
print(expected)

Chi-squared: 2.3070394038735307
P-value: 0.12878914467962418
Degrees of freedom: 1
Expected frequencies:
[[ 50.54748603  27.45251397]
 [413.45251397 224.54748603]]


In [150]:
if p_val > alpha:
  print('Нулевая гипотеза верна')
else:
  print('Отклоняем нулевую гипотезу')

Нулевая гипотеза верна


P-value больше уровня статзначимости, значит нет оснований отклонять нулевую гипотезу о том, что ушедшие и неушедшие компании равны по численности сотрудников

### Изменение цены тарифа

Формулируем гипотезы:

>$H_0$ - Ушедшие и неушедшие компании равны по фактору изменения цены тарифа
>
>$H_1$ - Ушедшие и неушедшие компании не равны по фактору изменения цены тарифа
>

In [151]:
pv = billing_companies[['company_id', 'is_active', 'unique_price_ids']]
pv['is_one_tariff_price_id'] = pv.unique_price_ids.apply(lambda x: 1 if x == 1 else 0)
pv = pd.crosstab(pv['is_one_tariff_price_id'], pv['is_active'])
pv

is_active,False,True
is_one_tariff_price_id,,
0,258,155
1,1141,287


In [152]:
# Тест хи-квадрат
chi2, p_val, dof, expected = chi2_contingency(pv)

In [153]:
# Выведем результаты теста
print(f"Chi-squared: {chi2}")
print(f"P-value: {p_val}")
print(f"Degrees of freedom: {dof}")
print("Expected frequencies:")
print(expected)

Chi-squared: 52.40658769691491
P-value: 4.5120782401400365e-13
Degrees of freedom: 1
Expected frequencies:
[[ 313.84410646   99.15589354]
 [1085.15589354  342.84410646]]


In [154]:
if p_val > alpha:
  print('Нулевая гипотеза верна')
else:
  print('Отклоняем нулевую гипотезу')

Отклоняем нулевую гипотезу


P-value меньше уровня статзначимости, отклоняем нулевую гипотезу о том, что ушедшие и неушедшие компании равны по фактору изменения цены тарифа

### Старт: id оплаты 23

Формулируем гипотезы:

>$H_0$ - Внутри тарифа Старт с id оплаты 23, и других id оплаты компании уходят одинаково
>
>$H_1$ - Внутри тарифа Старт с id оплаты 23, и других id оплаты компании уходят не одинаково
>

In [155]:
pv = billing_companies.query('tariff == "Старт"')
pv['is_billing_tariff_price_id_23'] = pv.billing_tariff_price_id.apply(lambda x: 1 if x == '23' else 0)
pv = pd.crosstab(pv['is_billing_tariff_price_id_23'], pv['is_active'])
pv

is_active,False,True
is_billing_tariff_price_id_23,,
0,332,66
1,237,87


In [156]:
# Тест хи-квадрат
chi2, p_val, dof, expected = chi2_contingency(pv)

In [157]:
# Выведем результаты теста
print(f"Chi-squared: {chi2}")
print(f"P-value: {p_val}")
print(f"Degrees of freedom: {dof}")
print("Expected frequencies:")
print(expected)

Chi-squared: 10.670986698127031
P-value: 0.0010882905002671203
Degrees of freedom: 1
Expected frequencies:
[[313.65927978  84.34072022]
 [255.34072022  68.65927978]]


In [158]:
if p_val > alpha:
  print('Нулевая гипотеза верна')
else:
  print('Отклоняем нулевую гипотезу')

Отклоняем нулевую гипотезу


P-value меньше уровня статзначимости, отклоняем нулевую гипотезу о том, что внутри тарифа Старт с id оплаты 23, и других id оплаты компании уходят одинаково

### Базовый: id оплаты 24

Формулируем гипотезы:

>$H_0$ - Внутри тарифа Базовый с id оплаты 24, и других id оплаты компании уходят одинаково
>
>$H_1$ - Внутри тарифа Базовый с id оплаты 24, и других id оплаты компании уходят не одинаково
>

In [159]:
pv = billing_companies.query('tariff == "Базовый"')
pv['is_billing_tariff_price_id_24'] = pv.billing_tariff_price_id.apply(lambda x: 1 if x == '24' else 0)
pv = pd.crosstab(pv['is_billing_tariff_price_id_24'], pv['is_active'])
pv

is_active,False,True
is_billing_tariff_price_id_24,,
0,447,133
1,217,76


In [160]:
# Тест хи-квадрат
chi2, p_val, dof, expected = chi2_contingency(pv)

In [161]:
# Выведем результаты теста
print(f"Chi-squared: {chi2}")
print(f"P-value: {p_val}")
print(f"Degrees of freedom: {dof}")
print("Expected frequencies:")
print(expected)

Chi-squared: 0.8088622972907734
P-value: 0.3684568267425685
Degrees of freedom: 1
Expected frequencies:
[[441.14547537 138.85452463]
 [222.85452463  70.14547537]]


In [162]:
if p_val > alpha:
  print('Нулевая гипотеза верна')
else:
  print('Отклоняем нулевую гипотезу')

Нулевая гипотеза верна


P-value больше уровня статзначимости, значит нет оснований отклонять нулевую гипотезу о том, что внутри тарифа Базовый с id оплаты 24, и других id оплаты компании уходят одинаково

## Группы : Компании с лайфтаймом > и <= 18 месяцев

In [163]:
companies_lifetime['is_lt_more_18'] = companies_lifetime.subscription_period_months.apply(lambda x: 1 if x > 18 else 0)

### Среднее количество активностей

Формулируем гипотезы:

>$H_0$ - Компании с лайфтаймом больше 18 месяцев и остальные равны по количеству активностей
>
>$H_1$ - Компании с лайфтаймом больше 18 месяцев и остальные не равны по количеству активностей
>

In [164]:
pv = companies_by_mean_activity[['company_id', 'mean_number_of_activities']].merge(companies_lifetime[['company_id', 'is_lt_more_18']], on='company_id', how='inner')

In [165]:
group_a = pv.query('is_lt_more_18 == 1')['mean_number_of_activities']
group_b = pv.query('is_lt_more_18 == 0')['mean_number_of_activities']

In [166]:
equal_var(group_a, group_b)

Student’s t-test


In [167]:
# T-тест между двумя группами
t_stat, p_val = ttest_ind(group_a, group_b, equal_var=True)

print(f"Значение t-статистики: {t_stat}")
print(f"P-value для t-теста: {p_val}")
print('------------')

if p_val > alpha:
  print('Нулевая гипотеза верна')
else:
  print('Отклоняем нулевую гипотезу')

Значение t-статистики: 2.738890338324398
P-value для t-теста: 0.006323349378558845
------------
Отклоняем нулевую гипотезу


P-value очень мало, отклоняем нулевую гипотезу о том, что компании с лайфтаймом больше 18 месяцев и остальные равны по количеству активностей

### Численность сотрудников

Формулируем гипотезы:

>$H_0$ - Компании с лайфтаймом больше 18 месяцев и остальные равны по численности сотрудников
>
>$H_1$ - Компании с лайфтаймом больше 18 месяцев и остальные не равны по численности сотрудников
>

In [168]:
#Пирсон x2
#группы сотрудники 1-10 и больше

In [169]:
pv = companies[['company_id', 'is_employers_less_11']].merge(companies_lifetime[['company_id', 'is_lt_more_18']], on='company_id', how='inner')

In [170]:
pv = pd.crosstab(pv['is_employers_less_11'], pv['is_lt_more_18'])
pv

is_lt_more_18,0,1
is_employers_less_11,,
0,35,39
1,388,232


In [171]:
# Тест хи-квадрат
chi2, p_val, dof, expected = chi2_contingency(pv)

In [172]:
# Выведем результаты теста
print(f"Chi-squared: {chi2}")
print(f"P-value: {p_val}")
print(f"Degrees of freedom: {dof}")
print("Expected frequencies:")
print(expected)

Chi-squared: 5.861742149623911
P-value: 0.015473471056200923
Degrees of freedom: 1
Expected frequencies:
[[ 45.1037464  28.8962536]
 [377.8962536 242.1037464]]


In [173]:
if p_val > alpha:
  print('Нулевая гипотеза верна')
else:
  print('Отклоняем нулевую гипотезу')

Отклоняем нулевую гипотезу


P-value меньше уровня статзначимости, отклоняем нулевую гипотезу о том, что компании с лайфтаймом больше 18 месяцев и остальные равны по численности сотрудников

### Изменение цены тарифа

Формулируем гипотезы:

>$H_0$ - Компании с лайфтаймом больше 18 месяцев и остальные равны по фактору изменения цены тарифа
>
>$H_1$ - Компании с лайфтаймом больше 18 месяцев и остальные не равны по фактору изменения цены тарифа
>

In [174]:
pv = billing_companies[['company_id', 'unique_price_ids']].merge(companies_lifetime[['company_id', 'is_lt_more_18']], on='company_id', how='inner')
pv['is_one_tariff_price_id'] = pv.unique_price_ids.apply(lambda x: 1 if x == 1 else 0)
pv = pd.crosstab(pv['is_one_tariff_price_id'], pv['is_lt_more_18'])
pv

is_lt_more_18,0,1
is_one_tariff_price_id,,
0,172,214
1,1050,370


In [175]:
# Тест хи-квадрат
chi2, p_val, dof, expected = chi2_contingency(pv)

In [176]:
# Выведем результаты теста
print(f"Chi-squared: {chi2}")
print(f"P-value: {p_val}")
print(f"Degrees of freedom: {dof}")
print("Expected frequencies:")
print(expected)

Chi-squared: 118.42677268831257
P-value: 1.398219461687558e-27
Degrees of freedom: 1
Expected frequencies:
[[261.18050941 124.81949059]
 [960.81949059 459.18050941]]


In [177]:
if p_val > alpha:
  print('Нулевая гипотеза верна')
else:
  print('Отклоняем нулевую гипотезу')

Отклоняем нулевую гипотезу


P-value меньше уровня статзначимости, отклоняем нулевую гипотезу о том, что компании с лайфтаймом больше 18 месяцев и остальные равны по фактору изменения цены тарифа

### Старт: id оплаты 23

Формулируем гипотезы:

>$H_0$ - Внутри тарифа Старт на id оплаты 23, и других id оплаты у компаний нет различий в лайфтайме
>
>$H_1$ - Внутри тарифа Старт на id оплаты 23, и других id оплаты у компаний есть различия в лайфтайме
>

In [178]:
pv = billing_companies[['company_id', 'tariff', 'billing_tariff_price_id']].query('tariff == "Старт"').merge(companies_lifetime[['company_id', 'is_lt_more_18']], on='company_id', how='inner')
pv['is_billing_tariff_price_id_23'] = pv.billing_tariff_price_id.apply(lambda x: 1 if x == '23' else 0)
pv = pd.crosstab(pv['is_billing_tariff_price_id_23'], pv['is_lt_more_18'])
pv

is_lt_more_18,0,1
is_billing_tariff_price_id_23,,
0,247,146
1,271,45


In [179]:
# Тест хи-квадрат
chi2, p_val, dof, expected = chi2_contingency(pv)

In [180]:
# Выведем результаты теста
print(f"Chi-squared: {chi2}")
print(f"P-value: {p_val}")
print(f"Degrees of freedom: {dof}")
print("Expected frequencies:")
print(expected)

Chi-squared: 45.55204942587243
P-value: 1.4863608065496497e-11
Degrees of freedom: 1
Expected frequencies:
[[287.12834979 105.87165021]
 [230.87165021  85.12834979]]


In [181]:
if p_val > alpha:
  print('Нулевая гипотеза верна')
else:
  print('Отклоняем нулевую гипотезу')

Отклоняем нулевую гипотезу


P-value меньше уровня статзначимости, отклоняем нулевую гипотезу о том, что внутри тарифа Старт на id оплаты 23, и других id оплаты у компаний нет различий в лайфтайме

### Базовый: id оплаты 24

Формулируем гипотезы:

>$H_0$ - Внутри тарифа Базовый на id оплаты 24, и других id оплаты у компаний нет различий в лайфтайме
>
>$H_1$ - Внутри тарифа Базовый на id оплаты 24, и других id оплаты у компаний есть различия в лайфтайме
>

In [182]:
pv = billing_companies[['company_id', 'tariff', 'billing_tariff_price_id']].query('tariff == "Базовый"').merge(companies_lifetime[['company_id', 'is_lt_more_18']], on='company_id', how='inner')
pv['is_billing_tariff_price_id_24'] = pv.billing_tariff_price_id.apply(lambda x: 1 if x == '24' else 0)
pv = pd.crosstab(pv['is_billing_tariff_price_id_24'], pv['is_lt_more_18'])
pv

is_lt_more_18,0,1
is_billing_tariff_price_id_24,,
0,367,199
1,189,101


In [183]:
# Тест хи-квадрат
chi2, p_val, dof, expected = chi2_contingency(pv)

In [184]:
# Выведем результаты теста
print(f"Chi-squared: {chi2}")
print(f"P-value: {p_val}")
print(f"Degrees of freedom: {dof}")
print("Expected frequencies:")
print(expected)

Chi-squared: 0.00042070654430522813
P-value: 0.9836356317303938
Degrees of freedom: 1
Expected frequencies:
[[367.63551402 198.36448598]
 [188.36448598 101.63551402]]


In [185]:
if p_val > alpha:
  print('Нулевая гипотеза верна')
else:
  print('Отклоняем нулевую гипотезу')

Нулевая гипотеза верна


P-value больше уровня статзначимости, значит нет оснований отклонять нулевую гипотезу о том, что внутри тарифа Базовый на id оплаты 24, и других id оплаты у компаний нет различий в лайфтайме

## Выводы по статтестам

**Сравнение групп ушедших и неушедших компаний**

Статтесты подтверждают, что у действующих компаний больше среднее значение активностей, чем у ушедших. Также, у ушедших компаний не менялся id оплаты тарифа. По численности сотрудников различий между группами нет.

Внутри тарифа Старт клиенты с id оплаты 23 уходят чаще, чем с других id.

Внутри тарифа Базовый клиенты с id оплаты 24 уходят не чаще, чем с других id.

**Сравнение групп компаний с лайфтаймом больше 18 месяцев, и <= 18 месяцев**

У компаний с лайфтаймом больше 18 месяцев больше среднее значение активностей. Также, у этих компаний менялся id оплаты тарифа и численность сотрудников больше 10.

Внутри тарифа Старт компании с лайфтаймом больше 18 месяцев не платят по id оплаты 23.

Внутри тарифа Базовый нет различий в id оплаты относительно лайфтайма компании.

# Сравнение ценовой политики сервиса с конкурентами


![](https://i.imgur.com/T4Z4xJl.png)

Самые популярные тарифы Старт и Базовый в Gectaro дороже, чем самые дорогие в других сервисах-конкурентах.

# Выводы и рекомендации

Основные тейки:
- качество данных - нет много компаний в companies, нет инфо о сфере деятельности
- обратная связь от  клиентов
- активности клиентов - чем больше активностей тем выше лайфтайм
- цены и тарифы - id оплаты, конкуренты
- изменение цены тарифа - если менять, то лайфтайм больше
- количество сотрудников -> привилегии малому бизнесу


**Итоги исследования**

В проекте по исследованию поведения пользователей имеются данные с начала 2019 года. Пользователь в данном случае - компания, пользующаяся сервисом и оплачивающая подписку на него.

За целевую переменную статуса компании Действующая/Ушедшая принят признак даты действия ее подписки -  если дата подписки на сервис меньше даты исследования, то компания считается прекратившей пользоваться сервисом, если больше - компания все еще действующий клиент.

В ходе исследования выполнено:
- Предобработка данных
- Основная информация о клиентах
- Отток клиентов
- Лайфтайм клиентов
- Составлен портрет клиента
- Информация об активностях
- Статистическими тестами проверены гипотезы на двух парах групп клиентов.
- Проведен мониторинг и сравнение цен с конкурентами

**Сложности в ходе исследования**

В данных о платежах `billing` количество уникальных компаний в 2.5 раза больше, чем в таблице с компаниями `companies`.

В таблице `people` `user_id` не привязан к уникальному пользователю.

Информация о сфере деятельности большинства клиентов отсутствует. Имеющиеся не имеют четкой структуры.

В `billing` нет сумм оплаты, что исключает возможность проведения RFM-анализа.

**Рекомендации заказчику**

***Качество данных***  
Стоит обратить внимание на количество и качество собираемых данных. Собирать полные данные о клиентах, применять категоризацию для информации о сфере деятельности клиента, унифицировать id пользователей.

***Обратная связь от клиентов***  
При прекращении компании пользоваться сервисом стоит собирать фидбек непосредственно от клиентов: их может не устраивать, например, качество самого сервиса, отсутствие определенных функций, работа поддержки или стоимость тарифа - по этим показателям у нас данных нет. Так как сервис работает по бизнес-модели b2b, и пользователей относительно немного, такие данные будут очень ценны.

***Активность клиентов***  
С увеличениеми среднемесячного количества активностей клиента в сервисе увеличивается его лайфтайм. Необходимо наладить работу с действующими, и будущими клиентами, совершающими мало активностей.

***Количество сотрудников***  
У компаний-клиентов, с количеством сотрудников меньше 11, в среднем меньше лайфтайм. Возможно, таким компаниям стоим предлагать персональную скидку, или другие бонусы, для увеличения их ужержания.

***Изменение цены тарифа***  
Факт изменения цены тарифа влияет на удержание пользователя. В ходе работы с клиентами нужно узнавать их мнение о цене тарифа, и, если они собираются уходить - делать скидку или предлагать более выгодные условия. Такая практика активно применяется у операторов сотовой связи.

***Цены и тарифы***  
При базовом сравнении цен сервиса Gectaro с конкурентами становится понятно, что у конкурентов цены значительно ниже. Стоит провести углубленное детальное исследование конкурентов с сравнением цен и предлагаемых услуг. В тарифе Старт стоит обратить внимание на id оплаты 23 - с него клиенты уходят чаще. В тарифе Базовый - на id оплаты 24 и 8.  Возможно, стоит делать некоторые привилегии или персональные предложения малому бизнесу - небольшим компаниям с количеством сотрудников до 10 и небольшим количеством активностей.